In [1]:
import pandas as pd
import numpy as np
import pathlib
import datetime
from datetime import datetime, timedelta, date
from rich import print
from collections import namedtuple
from dataclasses import dataclass, asdict
import operator
import tomllib
import tomlkit as tml
import json
import xarray as xr
import csvb.ingest as ci
import plotly.express as px
import plotly.graph_objects as go
#from decimal import Decimal, getcontext

#getcontext().prec = 2
pd.options.mode.copy_on_write = True

In [2]:
HOME = pathlib.Path.home()
cfgpth = HOME / ".config" / "csvb" / "config.toml"
with open(cfgpth, 'rb') as f:
    cfg = tomllib.load(f)
RULESPTH = HOME / cfg["paths"]["rules"]
BANKPTH = HOME / cfg["paths"]["bank"]
PRODUCTSPTH = HOME / cfg["paths"]["products"]

In [3]:
fd  = ci.find_bank_files(HOME / pathlib.Path(cfg["paths"]["bank"])) 

In [4]:
try:
    del apl
except NameError:
    pass
apl = ci.ingest_bank_files(fd["Liabilities:Apple:Credit Card"]["files"],
                           fd["Liabilities:Apple:Credit Card"],
                           RULESPTH)

apl = ci.read_and_apply(apl, RULESPTH / fd["Liabilities:Apple:Credit Card"]["rules_file"])
apl[apl["From"] == ""]

,Transaction Date,Clearing Date,Description,Merchant,Category,Type,Amount (USD),Purchased By,Amount,From


In [5]:
try:
    del apl_inc
except NameError:
    pass
apl_inc = ci.ingest_bank_files(fd["Liabilities:Apple:Installments"]["files"],
                           fd["Liabilities:Apple:Installments"],
                           RULESPTH)

apl_inc[apl_inc["From"] == ""]

,Date,Description,Amount,From


In [6]:
try:
    del fst
except NameError:
    pass
fst = ci.ingest_bank_files(fd["Assets:1st Source:Checking"]["files"],
                           fd["Assets:1st Source:Checking"],
                           RULESPTH)

fst = ci.read_and_apply(fst, RULESPTH / fd["Assets:1st Source:Checking"]["rules_file"])

fst[fst["From"] == ""]

,Transaction Number,Date,Description,Memo,Amount Debit,Amount Credit,Balance,Amount,From,Check Number,Fees


In [7]:
try:
    del ally
except NameError:
    pass
ally = ci.ingest_bank_files(fd["Assets:Ally:Savings"]["files"],
                           fd["Assets:Ally:Savings"],
                           RULESPTH)

ally = ci.read_and_apply(ally, RULESPTH / fd["Assets:Ally:Savings"]["rules_file"])

ally[ally["From"] == ""]

,Date,Time,Amount,Type,Description,From


In [8]:
try:
    del ndfcu
except NameError:
    pass
ndfcu = ci.ingest_bank_files(fd["Liabilities:NDFCU:Credit Card"]["files"],
                           fd["Liabilities:NDFCU:Credit Card"],
                           RULESPTH)

ndfcu = ci.read_and_apply(ndfcu, RULESPTH / fd["Liabilities:NDFCU:Credit Card"]["rules_file"])

ndfcu[ndfcu["From"] == ""]
ndfcu[pd.isnull(ndfcu["From"])]

,Date,Note,Description,Amount,Balance,From


In [9]:
bank_tables = {"Assets:1st Source:Checking": fst,
               "Liabilities:Apple:Credit Card": apl.rename(columns={"Transaction Date": "Date"}),
              "Assets:Ally:Savings": ally,
              "Liabilities:NDFCU:Credit Card": ndfcu,
              "Liabilities:Apple:Installments": apl_inc}


trans = ci.to_transactions(bank_tables)
trans = trans.sort_values("Date")
trans

,Date,Description,Amount,From,To
0,2021-12-31,MARTIN S SUPERMARKE . SOUTHBEND . IN 0000,-86.95,Expenses:Groceries,Liabilities:NDFCU:Credit Card
1,2021-12-31,PHILLIPS 66 - 0082 FIR . MISHAWAKA . IN 0000,-26.93,Expenses:Auto:Fuel,Liabilities:NDFCU:Credit Card
2,2021-12-31,"FANDANGO 12200 W. OLYMPIC BLVD., 8668575191 90...",-11.04,Expenses:Entertainment,Liabilities:Apple:Credit Card
3,2021-12-31,APPLE.COM/BILL ONE APPLE PARK WAY 866-712-7753...,-14.99,Expenses:Subscriptions,Liabilities:Apple:Credit Card
4,2022-01-01,CVS/PHARMACY #08642 . SOUTH BEND . IN 0000,-24.70,Expenses:Medical:Body,Liabilities:NDFCU:Credit Card
...,...,...,...,...,...
2559,2024-04-30,MONTHLY INSTALLMENTS (3 OF 12),-114.08,Liabilities:Apple:Installments,Liabilities:Apple:Credit Card
2555,2024-04-30,MONTHLY INSTALLMENTS (24 OF 24),-54.24,Liabilities:Apple:Installments,Liabilities:Apple:Credit Card
2554,2024-04-30,MONTHLY INSTALLMENTS (3 OF 12),-23.25,Liabilities:Apple:Installments,Liabilities:Apple:Credit Card
2556,2024-04-30,ND PAYROLL PAYROLL,5335.70,Income:Notre Dame:Regular,Assets:1st Source:Checking


In [11]:
try:
    del bal_decl
except NameError:
    pass

bal_decl = pd.read_csv(HOME / pathlib.Path(cfg["paths"]["bank"]) / "balances.csv",
                  header=0,
                  parse_dates=["Date"])

#bal_decl["From"] = ""
#bal["Amount"] = bal["Amount"].apply(strip_currency).astype(float)
#ndfcu = ally.rename(columns=dict(zip(ally.columns, [c.strip() for c in ally.columns])))
#bal_decl = bal_decl.sort_values("Date")
#bal_decl
#jan_bal = bal_decl.loc[(bal_decl["Date"] >= pd.Timestamp(2023, 12, 31)) &
#                       (bal_decl["Date"] <= pd.Timestamp(2024, 1, 1))
#trans = pd.concat([trans, bal_decl])
#bal_decl

In [12]:
try:
    del ledgers
except NameError:
    pass
    
#ledgers = create_ledgers(trans)
ledgers = {}
for acct in ci.accounts(trans):
    ledgers[acct] = ci.trans_to_ledger(trans, acct, bal_decl=bal_decl, clean=True)

#ledgers

In [ ]:
bankxr = 